# 🚀 SISTEMA OPERACIONAL DE MANTENIMIENTO PREDICTIVO - TIEMPO REAL

**Versión:** Operacional v1.0  
**Propósito:** Sistema para operación diaria en tiempo real  
**Base:** TFM_Sistema_Inferencia_Real.ipynb (validado)  

## 📋 CARACTERÍSTICAS OPERACIONALES
- ✅ Procesamiento de datos actuales/nuevos
- ✅ Detección automática de anomalías críticas
- ✅ Generación de OTs con umbrales calibrados
- ✅ Exportación automática de resultados
- ✅ Modo continuo para operación diaria
- ✅ Algoritmo de correspondencias matemático

---

In [ ]:
# 1. 🔧 CONFIGURACIÓN OPERACIONAL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pathlib import Path
import joblib
from datetime import datetime, timedelta
import uuid
import json

warnings.filterwarnings('ignore')

# 🎯 CONFIGURACIÓN DE RUTAS OPERACIONALES
BASE_PATH = Path(r'C:\TFM-pipeline')  # Ajustar según tu instalación
MODELO_PATH = BASE_PATH / 'modelo_predictivo_tfm.pkl'
INPUT_PATH = BASE_PATH / 'INPUT'
OUTPUT_PATH = BASE_PATH / 'output'
REPORTS_PATH = OUTPUT_PATH / 'reports'
OTS_PATH = OUTPUT_PATH / 'ots_generadas'

# Crear directorios necesarios
OUTPUT_PATH.mkdir(exist_ok=True)
REPORTS_PATH.mkdir(exist_ok=True)
OTS_PATH.mkdir(exist_ok=True)

print("🚀 SISTEMA OPERACIONAL DE MANTENIMIENTO PREDICTIVO")
print(f"📁 Modelo: {MODELO_PATH}")
print(f"📁 Datos: {INPUT_PATH}")
print(f"📁 Salida: {OUTPUT_PATH}")
print("=" * 60)

In [ ]:
# 2. 🔍 CARGAR MODELO PREDICTIVO
print("🔍 CARGANDO MODELO PREDICTIVO...")

try:
    datos_modelo = joblib.load(MODELO_PATH)
    modelo_cargado = datos_modelo['modelo_if']
    scaler_cargado = datos_modelo['scaler']
    variables_modelo = datos_modelo['variables']
    
    print("✅ Modelo: IsolationForest")
    print("✅ Scaler: StandardScaler")
    print(f"✅ Variables: {len(variables_modelo)} variables")
    print("✅ Compresor: C1")
    
    print(f"\n📋 Variables del modelo:")
    for i, var in enumerate(variables_modelo, 1):
        print(f"    {i:2d}. {var}")
        
except Exception as e:
    print(f"❌ Error cargando modelo: {e}")
    print("💡 Verificar que el archivo modelo_predictivo_tfm.pkl existe")

In [ ]:
# 3. 📊 FUNCIÓN DE CARGA DE DATOS OPERACIONAL
def cargar_datos_operacional(archivo_datos):
    """Carga y procesa datos para operación diaria"""
    
    print(f"📊 CARGANDO DATOS OPERACIONALES...")
    print(f"📁 Archivo: {archivo_datos}")
    
    try:
        # Cargar datos
        datos = pd.read_excel(archivo_datos)
        print(f"✅ Datos cargados: {len(datos):,} registros")
        
        # Procesar fechas
        if 'Hora' in datos.columns:
            datos['Hora'] = pd.to_datetime(datos['Hora'])
            datos.set_index('Hora', inplace=True)
            print(f"✅ Fechas procesadas: {datos.index.min()} a {datos.index.max()}")
        
        # Mapeo de variables (mismo que el sistema validado)
        mapeo_correcto = {
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total Actual - Fase A': 'THD_Corriente_A',
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total Actual - Fase B': 'THD_Corriente_B',
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total Actual - Fase C': 'THD_Corriente_C',
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total de Voltaje - Fase A': 'THD_Voltaje_A',
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total de Voltaje - Fase B': 'THD_Voltaje_B',
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total de Voltaje - Promedio': 'THD_Voltaje_Promedio',
            'Compresor C-1 - Calidad de la Energía: Distorsión Armónica Total de Voltaje - Fase C': 'THD_Voltaje_C',
            'Compresor C-1 - Calidad de la Energía: Demanda - Fase A (kW)': 'Potencia_A',
            'Compresor C-1 - Calidad de la Energía: Demanda - Fase B (kW)': 'Potencia_B',
            'Compresor C-1 - Calidad de la Energía: Demanda - Fase C (kW)': 'Potencia_C'
        }
        
        # Aplicar mapeo
        datos_mapeados = pd.DataFrame()
        variables_encontradas = 0
        
        for col_original, col_nueva in mapeo_correcto.items():
            if col_original in datos.columns:
                datos_mapeados[col_nueva] = datos[col_original]
                variables_encontradas += 1
        
        print(f"✅ Variables mapeadas: {variables_encontradas}/{len(mapeo_correcto)}")
        
        # Calcular THD promedio
        if all(col in datos_mapeados.columns for col in ['THD_Corriente_A', 'THD_Corriente_B', 'THD_Corriente_C', 'THD_Voltaje_Promedio']):
            datos_mapeados['THD'] = datos_mapeados[['THD_Corriente_A', 'THD_Corriente_B', 'THD_Corriente_C', 'THD_Voltaje_Promedio']].mean(axis=1)
            print("✅ THD promedio calculado")
        
        return datos_mapeados
        
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        return None

# Ejemplo de uso (ajustar nombre del archivo según necesidad)
archivo_datos = INPUT_PATH / 'datos_actuales.xlsx'  # Cambiar por el archivo actual
# datos_operacionales = cargar_datos_operacional(archivo_datos)

In [ ]:
# 4. 🔧 FUNCIONES DE DETECCIÓN CALIBRADAS
print("🔧 CONFIGURANDO FUNCIONES DE DETECCIÓN...")

def determinar_severidad(thd, score_anomalia):
    """Determina severidad con umbrales calibrados para operación"""
    if thd > 6.5:  # Umbral calibrado para críticos reales
        return 'CRÍTICO'
    elif thd > 2.0 or score_anomalia < -0.4:
        return 'ALERTA'
    elif thd > 1.5 or score_anomalia < -0.2:
        return 'ATENCIÓN'
    else:
        return 'MENOR'

def generar_prescripciones_especificas(severidad, thd, score_anomalia, timestamp):
    """Genera prescripciones específicas para operación"""
    
    prescripciones_base = {
        'CRÍTICO': [
            "🚨 PARADA INMEDIATA del compresor para inspección",
            "⚡ Verificación completa del sistema eléctrico trifásico",
            "🔍 Medición detallada de THD en todas las fases",
            "🛠️ Inspección de contactores y conexiones eléctricas",
            "📊 Análisis de armónicos con analizador de red",
            "🔧 Revisión de filtros armónicos si están instalados"
        ],
        'ALERTA': [
            "⚠️ Programar inspección eléctrica en próximas 8 horas",
            "⚡ Verificar conexiones y terminales eléctricos",
            "📈 Monitoreo continuo de THD por 24 horas",
            "🔍 Inspección visual de componentes eléctricos",
            "📊 Registro de parámetros eléctricos cada 2 horas"
        ],
        'ATENCIÓN': [
            "📋 Programar mantenimiento preventivo en 24 horas",
            "🔍 Inspección rutinaria del sistema eléctrico",
            "📊 Verificación de parámetros de calidad eléctrica",
            "🛠️ Limpieza de contactos y conexiones"
        ],
        'MENOR': [
            "📝 Inspección visual de rutina",
            "📊 Registro de parámetros operacionales",
            "🔍 Verificación de indicadores visuales"
        ]
    }
    
    if thd > 6.0:
        prescripciones_base[severidad].extend([
            "🚨 THD EXTREMO detectado - Riesgo crítico de daño a equipos",
            "⚡ Verificar inmediatamente cargas no lineales",
            "🔧 Instalación urgente de filtros armónicos requerida"
        ])
    
    return prescripciones_base.get(severidad, prescripciones_base['MENOR'])

def determinar_especialidad(severidad, thd):
    """Determina especialidad requerida"""
    if thd > 6.0 or severidad == 'CRÍTICO':
        return 'Eléctrica Senior'
    elif thd > 2.0 or severidad == 'ALERTA':
        return 'Eléctrica'
    elif severidad == 'ATENCIÓN':
        return 'Eléctrica/Mecánica'
    else:
        return 'Instrumentación'

def calcular_duracion_estimada(severidad):
    """Calcula duración estimada en horas"""
    duraciones = {
        'CRÍTICO': 8.0,
        'ALERTA': 4.0,
        'ATENCIÓN': 2.0,
        'MENOR': 1.0
    }
    return duraciones.get(severidad, 1.0)

def calcular_costo_estimado(severidad, duracion_horas):
    """Calcula costo estimado en USD"""
    costo_hora_especialista = {
        'CRÍTICO': 85.0,
        'ALERTA': 65.0,
        'ATENCIÓN': 45.0,
        'MENOR': 35.0
    }
    
    costo_mano_obra = duracion_horas * costo_hora_especialista.get(severidad, 35.0)
    costo_materiales = costo_mano_obra * 0.3
    
    return costo_mano_obra + costo_materiales

print("✅ Funciones de detección configuradas")

In [ ]:
# 5. 🚀 FUNCIÓN PRINCIPAL DE PROCESAMIENTO OPERACIONAL
def procesar_datos_tiempo_real(datos_mapeados, fecha_limite=None):
    """Procesa datos en tiempo real y genera resultados operacionales"""
    
    print("🚀 PROCESANDO DATOS EN TIEMPO REAL...")
    print("=" * 50)
    
    # Preparar datos para el modelo
    X_datos = datos_mapeados[variables_modelo].dropna()
    print(f"📊 Registros válidos: {len(X_datos):,}")
    
    if len(X_datos) == 0:
        print("❌ No hay datos válidos para procesar")
        return None, None
    
    # Aplicar modelo
    X_normalizado = scaler_cargado.transform(X_datos)
    predicciones = modelo_cargado.predict(X_normalizado)
    scores = modelo_cargado.decision_function(X_normalizado)
    
    print(f"✅ Modelo aplicado exitosamente")
    
    # Crear DataFrame de resultados
    resultados = pd.DataFrame({
        'fecha_deteccion': X_datos.index,
        'es_anomalia': predicciones == -1,
        'score_anomalia': scores,
        'THD': X_datos['THD'].values,
        'compresor': 'C1',
        'tipo_anomalia': 'THD_ELEVADO'
    })
    
    # Filtrar por fecha límite si se especifica
    if fecha_limite:
        resultados = resultados[resultados['fecha_deteccion'] <= fecha_limite]
        print(f"📅 Datos limitados hasta: {fecha_limite}")
    
    # Estadísticas
    total_anomalias = resultados['es_anomalia'].sum()
    porcentaje_anomalias = (total_anomalias / len(resultados)) * 100
    
    print(f"📊 Anomalías detectadas: {total_anomalias:,} ({porcentaje_anomalias:.2f}%)")
    print(f"📅 Período procesado: {resultados['fecha_deteccion'].min()} a {resultados['fecha_deteccion'].max()}")
    
    return resultados, X_datos

print("✅ Función de procesamiento configurada")

In [ ]:
# 6. 🔧 GENERADOR DE OTs OPERACIONAL
def generar_ots_operacional(resultados):
    """Genera OTs para operación diaria"""
    
    print("🔧 GENERANDO ÓRDENES DE TRABAJO...")
    print("=" * 40)
    
    # Filtrar anomalías
    anomalias_detectadas = resultados[resultados['es_anomalia']].copy()
    print(f"📊 Anomalías a procesar: {len(anomalias_detectadas):,}")
    
    if len(anomalias_detectadas) == 0:
        print("ℹ️ No se detectaron anomalías")
        return pd.DataFrame()
    
    ots_generadas = []
    fecha_max_datos = resultados['fecha_deteccion'].max()
    
    for idx, anomalia in anomalias_detectadas.iterrows():
        # Determinar severidad
        severidad = determinar_severidad(anomalia['THD'], anomalia['score_anomalia'])
        
        # Fecha de detección
        fecha_deteccion = pd.to_datetime(anomalia['fecha_deteccion'])
        
        # Calcular fecha programada
        if severidad == 'CRÍTICO':
            horas_respuesta = 2
        elif severidad == 'ALERTA':
            horas_respuesta = 8
        elif severidad == 'ATENCIÓN':
            horas_respuesta = 24
        else:
            horas_respuesta = 168
            
        fecha_programada = fecha_deteccion + timedelta(hours=horas_respuesta)
        
        # Limitar fecha programada al rango de datos
        if fecha_programada > fecha_max_datos:
            fecha_programada = fecha_max_datos
        
        # Generar prescripciones
        prescripciones = generar_prescripciones_especificas(
            severidad, anomalia['THD'], anomalia['score_anomalia'], fecha_deteccion
        )
        
        # Calcular parámetros
        especialidad = determinar_especialidad(severidad, anomalia['THD'])
        duracion = calcular_duracion_estimada(severidad)
        costo = calcular_costo_estimado(severidad, duracion)
        
        # Crear OT
        ot = {
            'numero_ot': f"OT-OP-{datetime.now().strftime('%Y%m%d')}-{len(ots_generadas)+1:04d}",
            'fecha_deteccion': fecha_deteccion,
            'fecha_programada': fecha_programada,
            'severidad': severidad,
            'tipo': 'CORRECTIVO' if severidad == 'CRÍTICO' else 'PREDICTIVO' if severidad == 'ALERTA' else 'PREVENTIVO',
            'prioridad': 1 if severidad == 'CRÍTICO' else 2 if severidad == 'ALERTA' else 3,
            'equipo': 'Compresor C1',
            'ubicacion': 'Sala de Compresores - Planta Refrigeración',
            'thd_detectado': anomalia['THD'],
            'score_anomalia': anomalia['score_anomalia'],
            'especialidad_requerida': especialidad,
            'duracion_estimada_horas': duracion,
            'costo_total_estimado': costo,
            'tiempo_respuesta': f'{horas_respuesta} horas',
            'prescripciones': prescripciones,
            'estado': 'PENDIENTE',
            'responsable': 'Jefe de Mantenimiento',
            'observaciones': f"Anomalía detectada por IA - THD: {anomalia['THD']:.3f}, Score: {anomalia['score_anomalia']:.3f}"
        }
        
        ots_generadas.append(ot)
    
    # Crear DataFrame
    ots_df = pd.DataFrame(ots_generadas)
    
    # Estadísticas
    print(f"✅ OTs generadas: {len(ots_df):,}")
    
    for severidad in ['CRÍTICO', 'ALERTA', 'ATENCIÓN', 'MENOR']:
        count = len(ots_df[ots_df['severidad'] == severidad])
        if count > 0:
            emoji = {'CRÍTICO': '🔴', 'ALERTA': '🟠', 'ATENCIÓN': '🟡', 'MENOR': '🔵'}.get(severidad, '⚪')
            print(f"   {emoji} {severidad}: {count} OTs")
    
    return ots_df

print("✅ Generador de OTs configurado")

In [ ]:
# 7. 📁 EXPORTADOR OPERACIONAL
def exportar_resultados_operacionales(ots_df, resultados, timestamp=None):
    """Exporta resultados en todos los formatos para operación"""
    
    if timestamp is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    print("📁 EXPORTANDO RESULTADOS OPERACIONALES...")
    print("=" * 45)
    
    archivos_generados = []
    
    if len(ots_df) > 0:
        # 1. Excel detallado
        try:
            archivo_excel = OTS_PATH / f'ots_operacional_{timestamp}.xlsx'
            
            with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
                # Hoja principal
                ots_df.to_excel(writer, sheet_name='OTs_Operacionales', index=False)
                
                # Hoja de críticas
                criticas = ots_df[ots_df['severidad'] == 'CRÍTICO']
                if len(criticas) > 0:
                    criticas.to_excel(writer, sheet_name='Críticas_Urgentes', index=False)
                
                # Hoja de alertas
                alertas = ots_df[ots_df['severidad'] == 'ALERTA']
                if len(alertas) > 0:
                    alertas.to_excel(writer, sheet_name='Alertas_Programadas', index=False)
                
                # Resumen ejecutivo
                resumen = pd.DataFrame({
                    'Tipo': ['Críticas', 'Alertas', 'Atención', 'Total'],
                    'Cantidad': [
                        len(criticas),
                        len(alertas), 
                        len(ots_df[ots_df['severidad'] == 'ATENCIÓN']),
                        len(ots_df)
                    ],
                    'Costo_Estimado_USD': [
                        criticas['costo_total_estimado'].sum() if len(criticas) > 0 else 0,
                        alertas['costo_total_estimado'].sum() if len(alertas) > 0 else 0,
                        ots_df[ots_df['severidad'] == 'ATENCIÓN']['costo_total_estimado'].sum(),
                        ots_df['costo_total_estimado'].sum()
                    ]
                })
                resumen.to_excel(writer, sheet_name='Resumen_Ejecutivo', index=False)
            
            archivos_generados.append(archivo_excel)
            print(f"✅ Excel: {archivo_excel.name}")
            
        except Exception as e:
            print(f"❌ Error Excel: {e}")
        
        # 2. CSV para integración
        try:
            archivo_csv = OTS_PATH / f'ots_operacional_{timestamp}.csv'
            ots_df.to_csv(archivo_csv, index=False, encoding='utf-8')
            archivos_generados.append(archivo_csv)
            print(f"✅ CSV: {archivo_csv.name}")
        except Exception as e:
            print(f"❌ Error CSV: {e}")
        
        # 3. JSON para APIs
        try:
            archivo_json = OTS_PATH / f'ots_operacional_{timestamp}.json'
            ots_df.to_json(archivo_json, orient='records', indent=2, date_format='iso')
            archivos_generados.append(archivo_json)
            print(f"✅ JSON: {archivo_json.name}")
        except Exception as e:
            print(f"❌ Error JSON: {e}")
        
        # 4. Reporte ejecutivo
        try:
            archivo_reporte = REPORTS_PATH / f'reporte_operacional_{timestamp}.md'
            
            reporte_md = f"""# 🚀 REPORTE OPERACIONAL - SISTEMA DE MANTENIMIENTO PREDICTIVO
**Fecha:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  
**Sistema:** Operacional Tiempo Real  
**Equipo:** Compresor C1  

## 📊 RESUMEN EJECUTIVO

### Datos Procesados
- **Total registros:** {len(resultados):,}
- **Período:** {resultados['fecha_deteccion'].min().strftime('%Y-%m-%d %H:%M')} a {resultados['fecha_deteccion'].max().strftime('%Y-%m-%d %H:%M')}
- **Anomalías detectadas:** {resultados['es_anomalia'].sum():,} ({(resultados['es_anomalia'].sum()/len(resultados)*100):.2f}%)

### Órdenes de Trabajo Generadas
- **Total OTs:** {len(ots_df)}
- **🔴 Críticas:** {len(ots_df[ots_df['severidad'] == 'CRÍTICO'])} (Respuesta: 2 horas)
- **🟠 Alertas:** {len(ots_df[ots_df['severidad'] == 'ALERTA'])} (Respuesta: 8 horas)
- **🟡 Atención:** {len(ots_df[ots_df['severidad'] == 'ATENCIÓN'])} (Respuesta: 24 horas)

### Costos Estimados
- **Total estimado:** ${ots_df['costo_total_estimado'].sum():,.2f} USD
- **Críticas:** ${ots_df[ots_df['severidad'] == 'CRÍTICO']['costo_total_estimado'].sum():,.2f} USD
- **Alertas:** ${ots_df[ots_df['severidad'] == 'ALERTA']['costo_total_estimado'].sum():,.2f} USD

## 🎯 ACCIONES INMEDIATAS REQUERIDAS

### 🚨 Críticas (Inmediato - 2 horas)
"""
            
            criticas = ots_df[ots_df['severidad'] == 'CRÍTICO']
            if len(criticas) > 0:
                for idx, ot in criticas.iterrows():
                    reporte_md += f"- **{ot['numero_ot']}:** THD {ot['thd_detectado']:.2f} - {ot['fecha_deteccion'].strftime('%Y-%m-%d %H:%M')}\n"
            else:
                reporte_md += "- No hay OTs críticas\n"
            
            reporte_md += f"""
### ⚠️ Alertas (8 horas)
"""
            
            alertas = ots_df[ots_df['severidad'] == 'ALERTA']
            if len(alertas) > 0:
                for idx, ot in alertas.head(5).iterrows():  # Mostrar solo las primeras 5
                    reporte_md += f"- **{ot['numero_ot']}:** THD {ot['thd_detectado']:.2f} - {ot['fecha_deteccion'].strftime('%Y-%m-%d %H:%M')}\n"
                if len(alertas) > 5:
                    reporte_md += f"- ... y {len(alertas)-5} alertas adicionales\n"
            else:
                reporte_md += "- No hay alertas pendientes\n"
            
            reporte_md += f"""
## 📋 RECOMENDACIONES OPERACIONALES

1. **Priorizar OTs críticas** en las próximas 2 horas
2. **Programar alertas** dentro de 8 horas
3. **Monitorear continuamente** el sistema para nuevas anomalías
4. **Actualizar estado** de OTs completadas en el sistema

---
*Reporte generado automáticamente por el Sistema Operacional de Mantenimiento Predictivo*
*Timestamp: {timestamp}*
"""
            
            with open(archivo_reporte, 'w', encoding='utf-8') as f:
                f.write(reporte_md)
            
            archivos_generados.append(archivo_reporte)
            print(f"✅ Reporte: {archivo_reporte.name}")
            
        except Exception as e:
            print(f"❌ Error reporte: {e}")
    
    else:
        print("ℹ️ No hay OTs para exportar")
    
    print(f"\n📂 Archivos generados: {len(archivos_generados)}")
    print(f"📁 Ubicación: {OTS_PATH}")
    
    return archivos_generados

print("✅ Exportador configurado")

In [ ]:
# 8. 🔄 FUNCIÓN PRINCIPAL OPERACIONAL
def ejecutar_sistema_operacional(archivo_datos, fecha_limite=None):
    """Función principal para ejecutar el sistema operacional completo"""
    
    print("🔄 INICIANDO SISTEMA OPERACIONAL COMPLETO...")
    print("=" * 60)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    try:
        # 1. Cargar datos
        datos_mapeados = cargar_datos_operacional(archivo_datos)
        if datos_mapeados is None:
            return False
        
        # 2. Procesar datos
        resultados, X_datos = procesar_datos_tiempo_real(datos_mapeados, fecha_limite)
        if resultados is None:
            return False
        
        # 3. Generar OTs
        ots_df = generar_ots_operacional(resultados)
        
        # 4. Exportar resultados
        archivos_generados = exportar_resultados_operacionales(ots_df, resultados, timestamp)
        
        # 5. Resumen final
        print("\n🎯 RESUMEN FINAL OPERACIONAL")
        print("=" * 40)
        print(f"✅ Registros procesados: {len(resultados):,}")
        print(f"✅ Anomalías detectadas: {resultados['es_anomalia'].sum():,}")
        print(f"✅ OTs generadas: {len(ots_df)}")
        
        if len(ots_df) > 0:
            criticas = len(ots_df[ots_df['severidad'] == 'CRÍTICO'])
            alertas = len(ots_df[ots_df['severidad'] == 'ALERTA'])
            
            print(f"🔴 OTs críticas: {criticas} (Atención inmediata)")
            print(f"🟠 OTs alertas: {alertas} (Programar en 8h)")
            print(f"💰 Costo total estimado: ${ots_df['costo_total_estimado'].sum():,.2f} USD")
        
        print(f"📁 Archivos generados: {len(archivos_generados)}")
        print(f"📂 Ubicación: {OTS_PATH}")
        print(f"🕐 Timestamp: {timestamp}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error en sistema operacional: {e}")
        return False

print("✅ Sistema operacional configurado")

## 🚀 EJECUCIÓN OPERACIONAL

### Instrucciones de Uso:

1. **Preparar datos:** Colocar archivo Excel en la carpeta `INPUT/`
2. **Ajustar nombre:** Cambiar `nombre_archivo.xlsx` por el archivo real
3. **Ejecutar:** Correr la celda siguiente
4. **Revisar resultados:** Verificar archivos generados en `output/ots_generadas/`

### Archivos de Salida:
- **Excel:** OTs detalladas con múltiples hojas
- **CSV:** Para integración con otros sistemas
- **JSON:** Para APIs y servicios web
- **Reporte MD:** Resumen ejecutivo en Markdown

---

In [ ]:
# 9. 🎯 EJECUCIÓN OPERACIONAL
# AJUSTAR EL NOMBRE DEL ARCHIVO SEGÚN NECESIDAD

# Ejemplo para datos actuales
archivo_datos_actual = INPUT_PATH / 'datos_actuales.xlsx'  # CAMBIAR POR EL ARCHIVO REAL

# Ejemplo para datos de un período específico
# archivo_datos_actual = INPUT_PATH / 'InformacionSeptiembre_fp1.xlsx'
# fecha_limite = pd.Timestamp('2025-09-30')  # Opcional: limitar hasta una fecha

# EJECUTAR SISTEMA OPERACIONAL
if archivo_datos_actual.exists():
    print(f"📁 Procesando archivo: {archivo_datos_actual.name}")
    
    # Ejecutar sistema completo
    exito = ejecutar_sistema_operacional(
        archivo_datos_actual,
        fecha_limite=None  # Cambiar por fecha específica si es necesario
    )
    
    if exito:
        print("\n🎉 SISTEMA OPERACIONAL EJECUTADO EXITOSAMENTE")
        print("📋 Revisar archivos generados para acciones inmediatas")
    else:
        print("\n❌ Error en la ejecución del sistema operacional")
        
else:
    print(f"❌ Archivo no encontrado: {archivo_datos_actual}")
    print("💡 Verificar que el archivo existe en la carpeta INPUT/")
    print("💡 Ajustar el nombre del archivo en la variable 'archivo_datos_actual'")

## 🔄 MODO CONTINUO (OPCIONAL)

Para operación continua, se puede programar la ejecución automática:

### Opciones de Automatización:
1. **Cron Job (Linux/Mac):** Ejecutar cada hora/día
2. **Task Scheduler (Windows):** Programar ejecución automática
3. **Script Python:** Loop con sleep para ejecución periódica
4. **Jupyter Scheduler:** Extensiones para programar notebooks

### Ejemplo de Script Continuo:

In [ ]:
# 10. 🔄 MODO CONTINUO (DESCOMENTA PARA USAR)
# ADVERTENCIA: Solo usar en entorno de producción controlado

"""
import time
import schedule

def job_operacional():
    """Job para ejecución programada"""
    print(f"🔄 Ejecutando job operacional: {datetime.now()}")
    
    # Buscar archivos nuevos en INPUT/
    archivos_nuevos = list(INPUT_PATH.glob('*.xlsx'))
    
    for archivo in archivos_nuevos:
        if 'procesado' not in archivo.name.lower():
            print(f"📁 Procesando: {archivo.name}")
            
            exito = ejecutar_sistema_operacional(archivo)
            
            if exito:
                # Marcar como procesado
                nuevo_nombre = archivo.parent / f"procesado_{archivo.name}"
                archivo.rename(nuevo_nombre)
                print(f"✅ Archivo procesado y renombrado")

# Programar ejecución cada hora
# schedule.every().hour.do(job_operacional)

# Programar ejecución diaria a las 08:00
# schedule.every().day.at("08:00").do(job_operacional)

# Loop continuo
# while True:
#     schedule.run_pending()
#     time.sleep(60)  # Verificar cada minuto
"""

print("ℹ️ Modo continuo disponible (descomenta para activar)")
print("⚠️ Solo usar en entorno de producción controlado")

## 📋 NOTAS OPERACIONALES

### ✅ Funcionalidades Validadas:
- Detección de anomalías con umbrales calibrados
- Generación de OTs con severidad correcta
- Exportación multi-formato (Excel, CSV, JSON)
- Reportes ejecutivos automáticos
- Cálculo de costos y tiempos de respuesta

### 🔧 Configuraciones Importantes:
- **Umbrales críticos:** THD > 6.5 (calibrado para detección real)
- **Tiempos de respuesta:** Crítico=2h, Alerta=8h, Atención=24h
- **Especialidades:** Automáticas según severidad y THD
- **Costos:** Calculados según especialidad y duración

### 📁 Estructura de Archivos:
```
C:/TFM-pipeline/
├── INPUT/                     # Archivos de datos
├── output/
│   ├── ots_generadas/        # OTs exportadas
│   └── reports/              # Reportes ejecutivos
├── modelo_predictivo_tfm.pkl # Modelo entrenado
└── TFM_Sistema_Operacional_Tiempo_Real.ipynb
```

### 🚀 Para Uso Diario:
1. Colocar archivo de datos en `INPUT/`
2. Ajustar nombre en celda de ejecución
3. Ejecutar notebook completo
4. Revisar archivos en `output/ots_generadas/`
5. Actuar sobre OTs críticas inmediatamente

---
**Sistema Operacional de Mantenimiento Predictivo v1.0**  
*Basado en TFM_Sistema_Inferencia_Real.ipynb validado*